In [1]:
import sys  
sys.path.insert(0, '../')

from connection_to_db import connection

In [2]:
sys.path.insert(0, '/')

from preprocess import preprocess as preprocess_data

In [3]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [8]:
departures_test_name = 'departures_to_test'
departures_train_name = 'departures'

weather_test_name = 'weather_predict'
weather_train_name = 'weather'

In [5]:
_, conn, cur = connection()

In [6]:
# to look up existing tables

#cur.execute(
#"""SELECT table_name
#FROM INFORMATION_SCHEMA.TABLES
#WHERE table_type = 'BASE TABLE'"""
#    )
#cur.fetchall()

In [12]:
def get_data(connection, table_name, weather_table_name, is_train=False):
    query = f'''
        SELECT * 
        FROM {table_name}
        LEFT JOIN {weather_table_name} ON {weather_table_name}.datetime = {table_name}.datetime_round
    '''
    if is_train:
        query += ' WHERE delay IS NOT NULL'
    data = pd.read_sql(query, connection)
    return data

In [14]:
test = get_data(conn, departures_test_name, weather_test_name)
test = preprocess_data(test)
test.head()

,carrier_code,flight,destination_airport,scheduled_datetime,date,delay,temp,humidity,precip,preciptype,...,cloudcover,visibility,solarradiation,conditions,winspeed_greater_17,is_freezing_rain,is_first_day_half,weekday,is_weekend,season
0,51,7454,8,00:01:00,2023-01-19,None,38.0,88.6,0.4,4,...,100.0,1.6,0.0,0,1,0,False,3,False,3
1,51,7452,8,00:01:00,2023-01-19,None,38.0,88.6,0.4,4,...,100.0,1.6,0.0,0,1,0,False,3,False,3
2,51,7458,8,00:01:00,2023-01-19,None,38.0,88.6,0.4,4,...,100.0,1.6,0.0,0,1,0,False,3,False,3
3,51,7456,8,00:01:00,2023-01-19,None,38.0,88.6,0.4,4,...,100.0,1.6,0.0,0,1,0,False,3,False,3
4,16,6444,110,00:01:00,2023-01-19,None,38.0,88.6,0.4,4,...,100.0,1.6,0.0,0,1,0,False,3,False,3


In [17]:
train = get_data(conn, departures_train_name, weather_train_name)
train = preprocess_data(train, is_data_train=True)
train.head()

,carrier_code,flight,destination_airport,scheduled_datetime,date,delay,temp,humidity,precip,preciptype,...,cloudcover,visibility,solarradiation,conditions,winspeed_greater_17,is_freezing_rain,is_first_day_half,weekday,is_weekend,season
0,68,5429,50,11:50:00,2022-03-01,0.0,46.3,59.9,0.0,2,...,100.0,9.9,156.0,5,0,0,True,1,False,3
1,68,5433,245,20:30:00,2022-03-01,0.0,36.1,78.5,0.0,2,...,86.3,9.9,0.0,6,0,0,False,1,False,3
2,68,5446,50,13:58:00,2022-03-01,0.0,51.2,50.0,0.0,2,...,85.6,9.9,377.0,6,0,0,True,1,False,3
3,68,5447,128,11:15:00,2022-03-01,19.0,46.3,59.9,0.0,2,...,100.0,9.9,156.0,5,0,0,True,1,False,3
4,68,5460,100,15:45:00,2022-03-01,0.0,48.8,56.3,0.0,2,...,81.8,9.9,483.0,6,0,0,True,1,False,3


In [19]:
def split_train(data):
    max_date = data['date'].max()
    train_old = data[data['date'] < max_date]
    train_new = data[data['date'] == max_date]
    return train_old, train_new

train_old, train_new = split_train(train)

In [20]:
class MeanModel:
    def __init__(self):
        self.summ = None
        self.train_size = None
    
    def train(self, data, key):
        self.summ = data[key].sum()
        self.train_size = len(data)
    
    def retrain(self, data, key):
        self.summ += data[key].sum()
        self.train_size += len(data)
    
    def predict(self, data):
        return [self.summ / self.train_size] * len(data)

In [21]:
mean_model = MeanModel()

In [22]:
mean_model.train(train_old, 'delay')

In [23]:
mean_model.retrain(train_new, 'delay')

In [24]:
predictions = mean_model.predict(test)
test['delay'] = predictions
pd.DataFrame(test).to_csv('preds_mean.csv', index=False, header=False)